# Lab | SQL Queries 9
In this lab, you will be using the Sakila database of movie rentals. You have been using this database for a couple labs already, but if you need to get the data again, refer to the official installation link.

### Instructions

* Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

    * Check the number of rentals for each customer for May.

    * Check the number of rentals for each customer for June.

      Hint: You can store the results from the two queries in two separate dataframes.
      

* Write a function that checks if customer borrowed more or less films in the month of June as compared to May.

    Hint: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the merge function.

In [106]:
import mysql.connector
import pandas as pd
import numpy as np

In [107]:
from mysql.connector import connect
from getpass import getpass

#### Connecting to SQL

In [109]:
db = connect(
    host = 'localhost',
    user = 'root',
    passwd = getpass('Enter password:'),
    database = 'sakila')

Enter password:········


#### Cursor is an instance used to execute SQL STATTEMENS on python

In [110]:
cursor = db.cursor()

#### Execute method

In [111]:
cursor.execute("SHOW DATABASES")

#### Fetching data

In [112]:
databases= cursor.fetchall()
print(databases)

[('bank',), ('bank_demo',), ('information_schema',), ('mysql',), ('performance_schema',), ('sakila',), ('sys',)]


## Check the number of rentals for each customer for May

In [113]:
# Defining the query.
query = "SELECT customer_id, COUNT(rental_id) AS number_rentals_may FROM rentals_may GROUP BY customer_id ORDER BY customer_id"
# Converting the data into pandas dataframe.
rentals_may = pd.read_sql_query(query, db)
rentals_may

/Users/pauli/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,number_rentals_may
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


## Check the number of rentals for each customer for June

In [114]:
# Defining the query.
query = "SELECT customer_id, COUNT(rental_id) AS number_rentals_june FROM rentals_june GROUP BY customer_id ORDER BY customer_id"
# Converting the data into pandas dataframe.
rentals_june = pd.read_sql_query(query, db)
rentals_june

/Users/pauli/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,number_rentals_june
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5
...,...,...
585,595,2
586,596,2
587,597,3
588,598,1


## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.

In [115]:
# Merge with Pandas.
rentals_may_june = pd.merge(rentals_may, rentals_june, how = 'outer', on = 'customer_id', sort = True)
rentals_may_june

,customer_id,number_rentals_may,number_rentals_june
0,1,2.0,7.0
1,2,1.0,1.0
2,3,2.0,4.0
3,4,NaN,6.0
4,5,3.0,5.0
...,...,...,...
593,595,1.0,2.0
594,596,6.0,2.0
595,597,2.0,3.0
596,598,NaN,1.0


In [116]:
# Checking the Nan values.
rentals_may_june.isnull().sum().sum()

86

In [117]:
# Filling the NaN with 0
rentals_may_june = rentals_may_june.fillna(0)
rentals_may_june

,customer_id,number_rentals_may,number_rentals_june
0,1,2.0,7.0
1,2,1.0,1.0
2,3,2.0,4.0
3,4,0.0,6.0
4,5,3.0,5.0
...,...,...,...
593,595,1.0,2.0
594,596,6.0,2.0
595,597,2.0,3.0
596,598,0.0,1.0


In [118]:
# Rename the columns.
rentals_may_june.rename(columns = {'number_rentals_x':'number_rentals_may', 'number_rentals_y':'number_rentals_june'}, inplace = True)
rentals_may_june

,customer_id,number_rentals_may,number_rentals_june
0,1,2.0,7.0
1,2,1.0,1.0
2,3,2.0,4.0
3,4,0.0,6.0
4,5,3.0,5.0
...,...,...,...
593,595,1.0,2.0
594,596,6.0,2.0
595,597,2.0,3.0
596,598,0.0,1.0


In [119]:
number_rentals = [
    (rentals_may_june['number_rentals_may'] > rentals_may_june['number_rentals_june']), 
    (rentals_may_june['number_rentals_may'] < rentals_may_june['number_rentals_june']), 
    (rentals_may_june['number_rentals_may']== rentals_may_june['number_rentals_june'])
]

options = ['May', 
           'June', 
           'Equal'
          ]

rentals_may_june['Customer borrowed more films'] = np.select(number_rentals, options)
rentals_may_june 

,customer_id,number_rentals_may,number_rentals_june,Customer borrowed more films
0,1,2.0,7.0,June
1,2,1.0,1.0,Equal
2,3,2.0,4.0,June
3,4,0.0,6.0,June
4,5,3.0,5.0,June
...,...,...,...,...
593,595,1.0,2.0,June
594,596,6.0,2.0,May
595,597,2.0,3.0,June
596,598,0.0,1.0,June
